In [1]:
import os
import logging
from typing import List, Tuple, Dict,Any
import sys
import numpy as np
import csv
import json
import sys
logger = logging.getLogger(__name__)

In [2]:
# ByteTrackのディレクトリのフルパスを指定します。例: '/home/username/ByteTrack'
byte_track_path = '/home/hayashi/YOLO_V8/ByteTrack'
if byte_track_path not in sys.path:
    sys.path.append(byte_track_path)

from ByteTrack.yolox.tracker.byte_tracker_with_scores import BYTETracker
from ByteTrack.yolox.tracker.byte_tracker_grid_test import BYTETrackerGrid

In [3]:
# ログ出力の設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# 1. camfusion_preprocessの出力csvを読み取る関数

logger = logging.getLogger(__name__)

import os
import csv
from typing import List, Dict
import logging

logger = logging.getLogger(__name__)

def load_csv_detections(csv_path: str, frame_number: int, format_type: str = "yolo") -> List[Dict]:
    """
    指定されたCSV形式から、特定フレームの検出データを読み込み、YOLO形式で返す。

    サポートされるCSVフォーマット:
    - "yolo": center_x_img, center_y_img, width_img, height_img
    - "pixel": x1, y1, w, h （すべてピクセル）

    :param csv_path: CSVファイルパス
    :param frame_number: 対象とするフレーム番号
    :param format_type: "yolo" または "pixel"
    :return: List of dicts in YOLO中心形式
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSVファイルが見つかりません: {csv_path}")

    detections: List[Dict] = []

    with open(csv_path, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                if int(row['frame']) != frame_number:
                    continue
            except (KeyError, ValueError):
                logger.warning("frame列の読み込みに失敗しました: %r", row)
                continue

            try:
                if format_type == "yolo":
                    det = {
                        "class_id": int(row['cls_id']),
                        "x_center": float(row['center_x_img']),
                        "y_center": float(row['center_y_img']),
                        "width":    float(row['width_img']),
                        "height":   float(row['height_img']),
                        "score":    float(row.get('confidence', 1.0))
                    }
                elif format_type == "pixel":
                    x1 = float(row['x1'])
                    y1 = float(row['y1'])
                    w  = float(row['w'])
                    h  = float(row['h'])
                    det = {
                        "class_id": int(row['cls_id']),
                        "x_center": x1 + w / 2,
                        "y_center": y1 + h / 2,
                        "width":    w,
                        "height":   h,
                        "score":    float(row.get('confidence', 1.0))
                    }
                else:
                    raise ValueError(f"Unsupported format_type: {format_type}")
                detections.append(det)
            except (KeyError, ValueError) as e:
                logger.warning("検出データ変換エラー (frame=%d): %s", frame_number, e)
                continue

    if not detections:
        logger.debug("フレーム%d の検出結果は空でした。", frame_number)
    return detections

In [5]:
# --- 2. YOLO形式から絶対座標への変換 ---
def convert_yolo_to_absolute(detections: List[Dict], img_width: int, img_height: int) -> List[Dict]:
    """
    YOLO検出結果（正規化座標）を、画像サイズに合わせた絶対座標に変換します。
    
    出力は以下の形式の辞書リストです:
      - bbox: (x1, y1, x2, y2)（ByteTrackなどに適した形式）
      - score: 検出信頼度
      - class_id: 検出対象のクラスID
    """
    abs_dets = []
    for det in detections:
        x_center_abs = det["x_center"] * img_width
        y_center_abs = det["y_center"] * img_height
        w_abs = det["width"] * img_width
        h_abs = det["height"] * img_height

        x1 = x_center_abs - w_abs / 2
        y1 = y_center_abs - h_abs / 2
        x2 = x_center_abs + w_abs / 2
        y2 = y_center_abs + h_abs / 2

        abs_dets.append({
            "bbox": [x1, y1, x2, y2],
            "score": det["score"],
            "class_id": det["class_id"]
        })
    return abs_dets

In [6]:
# --- 2'. pixelフォーマット用の絶対座標変換（BBox 作成） ---
def pixel_to_absolute_bbox(detections: List[Dict]) -> List[Dict]:
    """
    load_csv_detections(..., format_type="pixel") で返ってくる
    {'x_center','y_center','width','height','score','class_id'}
    から bytetrack 形式の 'bbox':[x1,y1,x2,y2] を作成します。
    """
    abs_dets = []
    for det in detections:
        x_c, y_c = det["x_center"], det["y_center"]
        w, h    = det["width"], det["height"]
        x1 = x_c - w/2
        y1 = y_c - h/2
        x2 = x_c + w/2
        y2 = y_c + h/2
        abs_dets.append({
            "bbox":    [x1, y1, x2, y2],
            "score":   det["score"],
            "class_id":det["class_id"]
        })
    return abs_dets


In [7]:
class ByteTrackWrapper:
    def __init__(self,
                 args,                    # BYTETrackerGrid にそのまま渡す args オブジェクト
                 frame_rate: int = 30     # フレームレート
                ):
        # BYTETrackerGrid の初期化
        self.tracker = BYTETracker(args, frame_rate)
        # logger.info("BYTETracker initialized with frame_rate=%d", frame_rate)
        # logger.info(" motion_lambda=%.3f, cost_alpha=%.3f",
        #             self.tracker.motion_lambda,
        #             self.tracker.cost_alpha)

    def update(self,
               detections: List[Dict],   # [{"bbox":[x1,y1,x2,y2],"score":s, ...}, ...]
               img_info: Tuple[int,int], # [orig_height, orig_width]
               img_size: Tuple[int,int]  # [model_input_h, model_input_w]
              ) -> Tuple[List[Dict], Dict]:
        """
        detections を BYTETrackerGrid.update の入力形式に変換し、
        出力の STrack リストを辞書のリストに戻します。

        Returns:
            tracks: [
                {
                  "track_id": int,
                  "bbox": [x1,y1,x2,y2],
                  "score": float,
                  "class_id": int
                }, ...
            ]
            stats: BYTETrackerGrid.update が返す統計情報の dict
        """
        # (1) 検出を [N,5] numpy array に
        if not detections:
            dets = np.zeros((0, 5), dtype=np.float32)
        else:
            dets = np.array(
                [[*det["bbox"], det.get("score", 1.0)] for det in detections],
                dtype=np.float32
            )

        # (2) ByteTracker で更新
        stracks, stats = self.tracker.update(dets, img_info, img_size)

        # (3) STrack → 辞書 に戻す
        results: List[Dict] = []
        for s in stracks:
            x, y, w, h = s.tlwh
            results.append({
                "track_id": int(s.track_id),
                "bbox":      [float(x), float(y), float(x + w), float(y + h)],
                "score":     float(s.score),
                "class_id":  0,  # 必要ならここをカスタマイズ
            })

        return results, stats

In [8]:
# --- 4. TrackEval形式で出力 ---
def write_trackeval_file(tracking_results_per_frame: List[Tuple[int, List[Dict]]],
                         output_filepath: str):
    """
    フレームごとのトラッキング結果からTrackEval形式のラベルファイルを生成し、出力します。

    出力例（CSV形式）：
      frame,track_id,x1,y1,w,h,score,class_id
    """
    
    with open(output_filepath, "w") as f:
        f.write("frame,track_id,x1,y1,w,h,score,class_id\n")
        for frame_number, tracks in tracking_results_per_frame:
            for track in tracks:
                x1, y1, x2, y2 = track["bbox"]
                w = x2 - x1
                h = y2 - y1
                line = f"{frame_number},{track['track_id']},{x1:.2f},{y1:.2f},{w:.2f},{h:.2f},{track['score']:.2f},{track['class_id']}\n"
                f.write(line)
    logger.info("TrackEvalファイルが %s に出力されました。", output_filepath)

In [9]:
def write_stats_csv(stats_list, stats_file):
    """
    フレームごとの stats 辞書のリストを CSV に書き出す。
    stats_list: [
        {'frame_id':1, 'total_detections':10, ...},
        {'frame_id':2, 'total_detections':12, ...},
         ...
    ]
    """
    if not stats_list:
        return

    # CSV のヘッダーを最初の辞書のキー順に固定
    fieldnames = [
        'frame_id',
        'total_detections',
        'high_confidence',
        'mid_confidence',
        'low_confidence',
        'high_unmatched',
        'mid_unmatched',
    ]
    with open(stats_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for stats in stats_list:
            writer.writerow(stats)

In [10]:
def compute_iou(boxA, boxB):
    """
    2つのバウンディングボックスの IoU を計算する。
      boxA, boxB: [x1, y1, x2, y2]
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0.0, xB - xA)
    interH = max(0.0, yB - yA)
    interArea = interW * interH

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = areaA + areaB - interArea

    return interArea / unionArea if unionArea > 0 else 0.0

def split_linked_by_iou(spawn_events, tracking_results_all, iou_thresh=0.3):
    """
    修正後: spawn_events 内の linked_pairs に old_frame/new_frame が含まれていることを前提。

    戻り値:
      with_iou_info: [
        {
          "old_id": int, "old_frame": int,
          "new_id": int, "new_frame": int,
          "overlap_ids": [int, …],
          "overlap_pairs": [
            { "old_id": int, "old_frame": int, "new_id": int, "new_frame": int }, …
          ]
        }, …
      ]
      without_iou_info: [
        {
          "old_id": int, "old_frame": int,
          "new_id": int, "new_frame": int
        }, …
      ]
    """

    # 1. frame → 全トラックリスト 辞書
    frame_to_all_tracks = {frame: tracks for frame, tracks in tracking_results_all}

    # 2. spawn_events から linked_pairs を集める
    all_linked = []
    for evt in spawn_events:
        for pair in evt.get("linked_pairs", []):
            all_linked.append({
                "old_id": pair["old_id"],
                "old_frame": pair["old_frame"],
                "new_id": pair["new_id"],
                "new_frame": pair["new_frame"]
            })

    with_iou_info = []
    without_iou_info = []

    # 3. 各リンクペアごとに消失時の IoU をチェック
    for pair in all_linked:
        old_id    = pair["old_id"]
        old_frame = pair["old_frame"]
        new_id    = pair["new_id"]
        new_frame = pair["new_frame"]

        # (a) 消失時の bbox を找出
        old_bbox = None
        for evt in spawn_events:
            if evt["frame"] == old_frame:
                for item in evt.get("disappeared_inside", []):
                    if item["track_id"] == old_id:
                        old_bbox = item["bbox"]
                        break
            if old_bbox is not None:
                break

        if old_bbox is None:
            without_iou_info.append({
                "old_id": old_id,
                "old_frame": old_frame,
                "new_id": new_id,
                "new_frame": new_frame
            })
            continue

        # (b) old_frame 時点の全トラックを取得
        all_tracks_at_old = frame_to_all_tracks.get(old_frame, [])

        # (c) IoU 判定
        overlap_ids = []
        for other in all_tracks_at_old:
            oid = other["track_id"]
            if oid == old_id:
                continue
            obbox = other["bbox"]

            xA = max(old_bbox[0], obbox[0])
            yA = max(old_bbox[1], obbox[1])
            xB = min(old_bbox[2], obbox[2])
            yB = min(old_bbox[3], obbox[3])

            interW = max(0.0, xB - xA)
            interH = max(0.0, yB - yA)
            interArea = interW * interH

            areaOld   = (old_bbox[2] - old_bbox[0]) * (old_bbox[3] - old_bbox[1])
            areaOther = (obbox[2] - obbox[0]) * (obbox[3] - obbox[1])
            unionArea = areaOld + areaOther - interArea

            iou = interArea / unionArea if unionArea > 0 else 0.0
            if iou >= iou_thresh:
                overlap_ids.append(oid)

        # (d) overlap_ids の有無で分類
        if overlap_ids:
            # IoUあり。さらに「overlap_idsの中にリンク済み old_id があれば、そのリンク先も overlap_pairs に追加」
            overlap_pairs = []
            for overlap_id in overlap_ids:
                # spawn_events の中から「linked_pairs」で old_id == overlap_id を探す
                found = False
                for evt_inner in spawn_events:
                    for pair_inner in evt_inner.get("linked_pairs", []):
                        if pair_inner["old_id"] == overlap_id:
                            overlap_pairs.append({
                                "old_id": pair_inner["old_id"],
                                "old_frame": pair_inner["old_frame"],
                                "new_id": pair_inner["new_id"],
                                "new_frame": pair_inner["new_frame"]
                            })
                            found = True
                            break
                    if found:
                        break

            with_iou_info.append({
                "old_id": old_id,
                "old_frame": old_frame,
                "new_id": new_id,
                "new_frame": new_frame,
                "overlap_ids": overlap_ids,
                "overlap_pairs": overlap_pairs
            })
        else:
            without_iou_info.append({
                "old_id": old_id,
                "old_frame": old_frame,
                "new_id": new_id,
                "new_frame": new_frame
            })

    return with_iou_info, without_iou_info

In [11]:
def detect_inside_events(appeared, disappeared, img_width, img_height, margin_ratio=0.03):
    """
    appeared: List of {"track_id": id, "bbox": [x1,y1,x2,y2]}
    disappeared: same format
    img_width, img_height: 画像サイズ
    margin_ratio: 端から除外する比率

    Returns:
      inside_appeared: List of appeared items whose center is in中央領域
      inside_disappeared: List of disappeared items whose center is in中央領域
    """
    margin_x = img_width * margin_ratio
    margin_y = img_height * margin_ratio

    inside_appeared = []
    for item in appeared:
        x1, y1, x2, y2 = item["bbox"]
        cx = (x1 + x2) / 2.0
        cy = (y1 + y2) / 2.0
        if (margin_x <= cx <= img_width - margin_x) and (margin_y <= cy <= img_height - margin_y):
            inside_appeared.append(item)

    inside_disappeared = []
    for item in disappeared:
        x1, y1, x2, y2 = item["bbox"]
        cx = (x1 + x2) / 2.0
        cy = (y1 + y2) / 2.0
        if (margin_x <= cx <= img_width - margin_x) and (margin_y <= cy <= img_height - margin_y):
            inside_disappeared.append(item)

    return inside_appeared, inside_disappeared

In [12]:
def detect_occlusions(inside_disappeared, prev_bbox_dict):
    """
    inside_disappeared: 中央領域で消えたアイテムのリスト
    prev_bbox_dict: 前フレームの {track_id: bbox}
    戻り値:
      occluded:     item["occluder_id"] を付与したリスト
      non_occluded: occluder 検出なしのリスト
    """
    occluded, non_occluded = [], []
    for item in inside_disappeared:
        x1, y1, x2, y2 = item["bbox"]
        occl = None
        for other_id, obb in prev_bbox_dict.items():
            if other_id == item["track_id"]:
                continue
            ox1, oy1, ox2, oy2 = obb
            if ox1 < x2 and ox2 > x1 and oy1 < y2 and oy2 > y1:
                occl = other_id
                break
        item["occluder_id"] = occl
        if occl is None:
            non_occluded.append(item)
        else:
            occluded.append(item)
    return occluded, non_occluded

In [13]:
def extract_occlusion_sessions(spawn_events: List[Dict]) -> List[Dict]:
    """
    spawn_events: run_tracking から得られるフレーム単位の辞書リスト。
      各要素は少なくとも以下のキーを持つ想定：
        - 'frame': int
        - 'occluded_disappeared': List[{'track_id': int, 'bbox': [...] }]
        - 'appeared_inside':    List[{'track_id': int, 'bbox': [...] }]
    戻り値はセッションのリスト。各セッションは dict で、
        - 'track_id':       occluded した対象のトラックID
        - 'start_frame':    遮蔽によって消えたフレーム番号
        - 'end_frame':      再出現した（中央領域内に現れた）フレーム番号
    """
    sessions = []
    active = {}  # track_id -> start_frame

    for evt in spawn_events:
        f = evt['frame']

        # --- セッション開始：遮蔽で消えた ---
        for item in evt.get('occluded_disappeared', []):
            tid = item['track_id']
            # すでに同じIDで進行中のセッションがなければ開始
            if tid not in active:
                active[tid] = f

        # --- セッション終了：中央領域内に再出現 ---
        for item in evt.get('appeared_inside', []):
            tid = item['track_id']
            if tid in active:
                start = active.pop(tid)
                sessions.append({
                    'track_id':    tid,
                    'start_frame': start,
                    'end_frame':   f
                })

    # --- ビデオの最後まで未解決のセッションは最終フレームで閉じる ---
    if spawn_events:
        last_frame = spawn_events[-1]['frame']
        for tid, start in active.items():
            sessions.append({
                'track_id':    tid,
                'start_frame': start,
                'end_frame':   last_frame
            })

    return sessions


In [14]:
def extract_occlusion_sessions(spawn_events: List[Dict[str, Any]]
                              ) -> List[Dict[str, Any]]:
    """
    spawn_events の連続するフレームデータから、
    ・消失イベント (occluded_disappeared + non_occluded_disappeared) をスタックに積み、
    ・生成イベント (appeared_inside) が来たら LIFO で pop、
    ・スタックが空になったら 1 セッションとしてまとめる

    戻り値の sessions は以下のキーを持つ dict のリスト:
      - start_frame: int  (最初の消失が起きたフレーム)
      - end_frame:   int  (最後の生成が起きたフレーム, スタック解消時)
      - disappeared: List[Tuple[int, int]]  # [(track_id, frame), …]
      - occluders:   List[Tuple[Optional[int], int]]  # [(occluder_id or None, frame), …]
      - generated:   List[Tuple[int, int]]  # [(new_track_id, frame), …]
      （後段で 'transients' が追加されます）
    """
    sessions: List[Dict[str, Any]] = []
    stack: List[Dict[str, Any]] = []
    current: Dict[str, Any] = None

    for ev in spawn_events:
        f = ev['frame']
        # --- 1) 消失イベントをスタックへ ---
        disappeared_all = ev.get('occluded_disappeared', []) + ev.get('non_occluded_disappeared', [])
        for item in disappeared_all:
            tid = item['track_id']
            occl = item.get('occluder_id')  # None なら非遮蔽
            # スタックに積む
            stack.append({'track_id': tid, 'occluder_id': occl, 'frame': f})
            # セッション開始
            if current is None:
                current = {
                    'start_frame': f,
                    'disappeared': [],  # List[(id, frame)]
                    'occluders': [],    # List[(occluder_id or None, frame)]
                    'generated': []     # List[(new_id, frame)]
                }
            current['disappeared'].append((tid, f))
            current['occluders'].append((occl, f))

        # --- 2) 生成イベントでスタックを LIFO pop ---
        for item in ev.get('appeared_inside', []):
            gen_id = item['track_id']
            if not stack:
                # pop すべき消失がない場合はスキップ
                continue
            popped = stack.pop()
            # セッションが未作成なら、消失情報を popped から初期化
            if current is None:
                current = {
                    'start_frame': popped['frame'],
                    'disappeared': [(popped['track_id'], popped['frame'])],
                    'occluders': [(popped['occluder_id'], popped['frame'])],
                    'generated': []
                }
            current['generated'].append((gen_id, f))

            # --- スタックが空になったらセッション終了 ---
            if not stack:
                current['end_frame'] = f
                sessions.append(current)
                current = None

    # --- 3) 最後まで解消しなかった場合、最終フレームで強制終了 ---
    if current is not None:
        last_f = spawn_events[-1]['frame']
        current['end_frame'] = last_f
        sessions.append(current)

    return sessions


In [15]:
def extract_transients(sessions, spawn_events):
    """
    sessions: extract_occlusion_sessions で得たリスト
    spawn_events: link_disappear_appear 後の spawn_events
    戻り値に、各セッションにおけるトランジェントID情報を追加します
    """
    for sess in sessions:
        start, end = sess["start_frame"], sess["end_frame"]
        # セッション期間中に出現したID→{id: first_appear_frame}
        appear_times = {}
        # 同じく消失したID→{id: disappear_frame}
        disappear_times = {}

        for evt in spawn_events:
            f = evt["frame"]
            if start < f <= end:
                # 出現レコードをチェック
                for item in evt.get("appeared_inside", []):
                    tid = item["track_id"]
                    if tid not in appear_times:
                        appear_times[tid] = f
                # 消失レコードをチェック
                for item in evt.get("disappeared_inside", []):
                    tid = item["track_id"]
                    # 出現後に消失したものだけキャッチ
                    if tid in appear_times and tid not in disappear_times:
                        disappear_times[tid] = f

        # トランジェントIDリストを作成
        sess["transients"] = [
            (tid, appear_times[tid], disappear_times[tid])
            for tid in disappear_times
        ]

    return sessions


In [16]:
def write_occlusion_sessions_csv(
    sessions: List[Dict[str, Any]],
    output_path: str
) -> None:
    """
    sessions: extract_occlusion_sessions ＋ extract_transients で得られたリスト
      各要素は dict で、キー:
        - start_frame, end_frame: int
        - disappeared, occluders, generated, transients: List[Tuple[int, int]]
    output_path: 出力先 CSV ファイルパス

    CSV のカラム:
      start_frame, end_frame, disappeared, occluders, generated, transients
    disappeared 等のリストは JSON 文字列として格納します。
    """
    fieldnames = [
        "start_frame",
        "end_frame",
        "disappeared",
        "occluders",
        "generated",
        "transients"
    ]
    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for sess in sessions:
            writer.writerow({
                "start_frame": sess["start_frame"],
                "end_frame":   sess["end_frame"],
                # 各リストは [(id, frame), …] の形なので JSON dump
                "disappeared": json.dumps(sess["disappeared"], ensure_ascii=False),
                "occluders":   json.dumps(sess["occluders"],   ensure_ascii=False),
                "generated":   json.dumps(sess["generated"],   ensure_ascii=False),
                "transients":  json.dumps(sess.get("transients", []), ensure_ascii=False),
            })

In [17]:
class DummyArgs:
    track_thresh = 0.5   # 例: 検出スコア0.5以上を主要候補とする
    track_buffer = 10    # 例: 30フレーム分をバッファとして許容する
    mot20 = False        # 例: MOT20用の設定でなく通常動作
    match_thresh = 0.99   # 例: マッチング閾値（IoUなど）を0.7に設定
    gate_factor = 1.0   # 動きによるゲーティング閾値に関与（上げると緩くなる）
    cost_alpha  = 0.1   # スコア計算に対する『IoU＋動き』VS『中心距離』の影響度に関与。0~1.0で、低いほど中心距離より

    w_iou = 0.0
    w_maha = 0.0
    w_center = 1.0
    use_motion_gating = False

args = DummyArgs()

In [18]:
def run_tracking(csv_path: str,
                 output_file: str,
                 stats_file: str,
                 img_width: int,
                 img_height: int,
                 frame_list: List[int],
                 tracker_params: Dict[str, Any],
                 margin_ratio: float = 0.03):
    """
    CSVの検出結果を使って ByteTrack で追跡を行い、遮蔽イベントを抽出してCSV出力します。

    :param csv_path:       detections.csv のパス
    :param output_file:    トラッキング結果出力パス
    :param stats_file:     遮蔽セッション出力CSVパス
    :param img_width:      画像幅
    :param img_height:     画像高さ
    :param frame_list:     フレーム番号のリスト
    :param tracker_params: ByteTrackWrapper 初期化用パラメータ
    :param margin_ratio:   中央領域判定用マージン比率
    
    :return: spawn_events, sessions
    """
    # トラッカー初期化
    tracker = ByteTrackWrapper(**tracker_params)

    prev_frame_ids = set()
    last_bbox_dict = {}
    spawn_events = []  # フレーム単位の生データ
    tracking_results_all = []

    # フレーム順処理
    for frame in frame_list:
        # (1) 検出読み込み
        detections = load_csv_detections(csv_path, frame, format_type = "pixel")
        
        #abs_dets = convert_yolo_to_absolute(detections, img_width, img_height)
        abs_dets = pixel_to_absolute_bbox(detections)

        # (2) トラッカー更新
        result = tracker.update(abs_dets, [img_height, img_width], [img_height, img_width])
        tracks = result[0] if isinstance(result, tuple) and len(result) >= 1 else result

        # (3) 出現・消失ID収集
        current_ids = {trk['track_id'] for trk in tracks}
        current_bboxes = {trk['track_id']: trk['bbox'] for trk in tracks}

        disappeared_ids = prev_frame_ids - current_ids
        appeared_ids   = current_ids - prev_frame_ids

        disappeared = [{ 'track_id': tid, 'bbox': last_bbox_dict[tid] }
                       for tid in disappeared_ids if tid in last_bbox_dict]
        appeared    = [{ 'track_id': tid, 'bbox': current_bboxes[tid] }
                       for tid in appeared_ids   if tid in current_bboxes]

        # (4) 中央領域判定
        inside_app, inside_dis = detect_inside_events(
            appeared, disappeared, img_width, img_height, margin_ratio
        )

        # (5) 遮蔽判定: 中央域で消えたものを occluded / non_occluded に分割
        occluded_dis, non_occluded_dis = detect_occlusions(
            inside_dis, last_bbox_dict
        )

        print(f"[DEBUG] frame={frame}: inside_disappeared={len(inside_dis)}")
        print(f"[DEBUG] frame={frame}: occluded={len(occluded_dis)}, non_occluded={len(non_occluded_dis)}")

        # (6) フレームデータ保存
        spawn_events.append({
            'frame': frame,
            'appeared': appeared,
            'disappeared': disappeared,
            'appeared_inside': inside_app,
            'disappeared_inside': inside_dis,
            'occluded_disappeared': occluded_dis,
            'non_occluded_disappeared': non_occluded_dis
        })

        prev_frame_ids = current_ids
        last_bbox_dict = current_bboxes.copy()
        tracking_results_all.append((frame, tracks))

    # (7) TrackEval 形式出力
    write_trackeval_file(tracking_results_all, output_file)
    #logger.info(f"TrackEval file written to {output_file}")

    # (8) セッション抽出
    sessions = extract_occlusion_sessions(spawn_events)
    # (9) トランジェント抽出（必要に応じて）
    sessions = extract_transients(sessions, spawn_events)

    # (10) セッションCSV出力
    write_occlusion_sessions_csv(sessions, stats_file)

    return spawn_events, sessions

In [22]:
import time

near_1_3_dict = {'00754-01276': [915, 1086], 
                '01943-02233': [1307, 1402], 
                '02987-03451': [1647, 1799], 
                '04060-04466': [2002, 2135], 
                '08323-08787': [3398, 3551], 
                '16298-16588': [6019, 6114], 
                '24708-25143': [8778, 8921], 
                '27405-27650': [9665, 9745]}

if __name__ == "__main__":
    # 結果を格納するリスト (任意)
    timing_results = []

    for i, filename in enumerate(near_1_3_dict):
        start_frame, end_frame = near_1_3_dict[filename]
        frame_list = list(range(start_frame, end_frame + 1))
        frame_count = len(frame_list)

        # 計測開始
        t0 = time.perf_counter()

        spawn_events = run_tracking(
            csv_path=f"system_output/datasets_up_to_lidar_tracking/tracking_and_processed_detection/near_1-3_iou_0.3/detection/{filename}.csv",
            output_file=f"official_time_test/raw_tracking_csv/{filename}.csv",
            stats_file=f"official_time_test/occlusion_session_csv/{filename}.csv",
            frame_list=frame_list,
            img_width=640,
            img_height=343,
            tracker_params={"args": args, "frame_rate": 10}
        )

        # 計測終了
        t1 = time.perf_counter()
        elapsed = t1 - t0

        # 結果を出力
        print(f"[{filename}] processed {frame_count} frames in {elapsed:.3f} s "
              f"({(elapsed/frame_count)*1000:.2f} ms/frame)")

        # 任意でまとめておきたい場合
        timing_results.append({
            "filename": filename,
            "frames": frame_count,
            "total_time_s": elapsed,
            "ms_per_frame": (elapsed/frame_count)*1000
        })

    # 最後にまとめて表示 (必要なら)
    print("\n=== Summary ===")
    for res in timing_results:
        print(f"{res['filename']}: {res['frames']} frames, "
              f"{res['total_time_s']:.3f} s total, "
              f"{res['ms_per_frame']:.2f} ms/frame")

[Frame 1] Total=14, High=12(Unmatched=12), Mid=2(Unmatched=2), Low=0
[DEBUG] frame=915: inside_disappeared=0
[DEBUG] frame=915: occluded=0, non_occluded=0
[Frame 2] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=916: inside_disappeared=0
[DEBUG] frame=916: occluded=0, non_occluded=0
[Frame 3] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=917: inside_disappeared=0
[DEBUG] frame=917: occluded=0, non_occluded=0
[Frame 4] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=918: inside_disappeared=0
[DEBUG] frame=918: occluded=0, non_occluded=0
[Frame 5] Total=14, High=13(Unmatched=2), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=919: inside_disappeared=1
[DEBUG] frame=919: occluded=0, non_occluded=1
[Frame 6] Total=12, High=12(Unmatched=2), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=920: inside_disappeared=1
[DEBUG] frame=920: occluded=0, non_occluded=1
[Frame 7] Total=12, High=12(Unmatched=2), Mid=0(Unmatched=0), Low=0
[DEBUG]

INFO:__main__:TrackEvalファイルが official_time_test/raw_tracking_csv/00754-01276.csv に出力されました。


[Frame 154] Total=11, High=11(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1068: inside_disappeared=0
[DEBUG] frame=1068: occluded=0, non_occluded=0
[Frame 155] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=1069: inside_disappeared=0
[DEBUG] frame=1069: occluded=0, non_occluded=0
[Frame 156] Total=11, High=11(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1070: inside_disappeared=0
[DEBUG] frame=1070: occluded=0, non_occluded=0
[Frame 157] Total=11, High=11(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1071: inside_disappeared=0
[DEBUG] frame=1071: occluded=0, non_occluded=0
[Frame 158] Total=11, High=11(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1072: inside_disappeared=0
[DEBUG] frame=1072: occluded=0, non_occluded=0
[Frame 159] Total=11, High=11(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1073: inside_disappeared=0
[DEBUG] frame=1073: occluded=0, non_occluded=0
[Frame 160] Total=10, High=10(Unmatched=0), Mid=0(Un

INFO:__main__:TrackEvalファイルが official_time_test/raw_tracking_csv/01943-02233.csv に出力されました。


[Frame 67] Total=12, High=10(Unmatched=0), Mid=2(Unmatched=0), Low=0
[DEBUG] frame=1373: inside_disappeared=0
[DEBUG] frame=1373: occluded=0, non_occluded=0
[Frame 68] Total=12, High=11(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=1374: inside_disappeared=0
[DEBUG] frame=1374: occluded=0, non_occluded=0
[Frame 69] Total=12, High=10(Unmatched=0), Mid=2(Unmatched=0), Low=0
[DEBUG] frame=1375: inside_disappeared=0
[DEBUG] frame=1375: occluded=0, non_occluded=0
[Frame 70] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=1376: inside_disappeared=1
[DEBUG] frame=1376: occluded=1, non_occluded=0
[Frame 71] Total=12, High=10(Unmatched=0), Mid=2(Unmatched=0), Low=0
[DEBUG] frame=1377: inside_disappeared=0
[DEBUG] frame=1377: occluded=0, non_occluded=0
[Frame 72] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=1378: inside_disappeared=0
[DEBUG] frame=1378: occluded=0, non_occluded=0
[Frame 73] Total=12, High=10(Unmatched=0), Mid=2(Unmatched

...

[Frame 43] Total=10, High=10(Unmatched=1), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=6061: inside_disappeared=0
[DEBUG] frame=6061: occluded=0, non_occluded=0
[Frame 44] Total=10, High=9(Unmatched=1), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=6062: inside_disappeared=0
[DEBUG] frame=6062: occluded=0, non_occluded=0
[Frame 45] Total=10, High=9(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=6063: inside_disappeared=0
[DEBUG] frame=6063: occluded=0, non_occluded=0
[Frame 46] Total=8, High=7(Unmatched=0), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=6064: inside_disappeared=3
[DEBUG] frame=6064: occluded=0, non_occluded=3
[Frame 47] Total=10, High=10(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=6065: inside_disappeared=0
[DEBUG] frame=6065: occluded=0, non_occluded=0
[Frame 48] Total=10, High=10(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=6066: inside_disappeared=0
[DEBUG] frame=6066: occluded=0, non_occluded=0
[Frame 49] Total=11, High=11(Unmatched=1), Mid=0(Unmatched=0),

INFO:__main__:TrackEvalファイルが official_time_test/raw_tracking_csv/24708-25143.csv に出力されました。


[Frame 131] Total=10, High=9(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=8908: inside_disappeared=0
[DEBUG] frame=8908: occluded=0, non_occluded=0
[Frame 132] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=8909: inside_disappeared=0
[DEBUG] frame=8909: occluded=0, non_occluded=0
[Frame 133] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=8910: inside_disappeared=0
[DEBUG] frame=8910: occluded=0, non_occluded=0
[Frame 134] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=8911: inside_disappeared=0
[DEBUG] frame=8911: occluded=0, non_occluded=0
[Frame 135] Total=10, High=10(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=8912: inside_disappeared=0
[DEBUG] frame=8912: occluded=0, non_occluded=0
[Frame 136] Total=11, High=10(Unmatched=0), Mid=1(Unmatched=1), Low=0
[DEBUG] frame=8913: inside_disappeared=0
[DEBUG] frame=8913: occluded=0, non_occluded=0
[Frame 137] Total=11, High=11(Unmatched=1), Mid=0(Unm

INFO:__main__:TrackEvalファイルが official_time_test/raw_tracking_csv/27405-27650.csv に出力されました。


[Frame 30] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=9694: inside_disappeared=0
[DEBUG] frame=9694: occluded=0, non_occluded=0
[Frame 31] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=9695: inside_disappeared=0
[DEBUG] frame=9695: occluded=0, non_occluded=0
[Frame 32] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=9696: inside_disappeared=0
[DEBUG] frame=9696: occluded=0, non_occluded=0
[Frame 33] Total=14, High=14(Unmatched=2), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=9697: inside_disappeared=0
[DEBUG] frame=9697: occluded=0, non_occluded=0
[Frame 34] Total=12, High=11(Unmatched=0), Mid=1(Unmatched=0), Low=0
[DEBUG] frame=9698: inside_disappeared=0
[DEBUG] frame=9698: occluded=0, non_occluded=0
[Frame 35] Total=12, High=12(Unmatched=0), Mid=0(Unmatched=0), Low=0
[DEBUG] frame=9699: inside_disappeared=0
[DEBUG] frame=9699: occluded=0, non_occluded=0
[Frame 36] Total=13, High=13(Unmatched=1), Mid=0(Unmatched